# Low Mach approximation Navier-Stokes element formulation and automatic differentiation
**Rubén Zorrilla**
UPC BarcelonaTech, CIMNE

## Governing equations
This formulation comes from a simplification of the fully compressible Navier-Stokes equation and is suitable for the resolution of low speed strongly thermally coupled flows.
Specifically, the formulation indroduces the low Mach limit simplification, which allows to split the total pressure into two parts, a thermodynamic part $p_{th}$ that is uniform in space and a mechanical (hydrodynamic) one $p$.
It is important to note that $p$ is several orders of magnitude smaller than $p_{th}$ and is therefore omitted from the state and energy equations.
In other words, the compressibility of the fluid comes from the thermal effects only.
Hence, the problem to be considered is finding a velocity field $\bm{u}$, mechanical pressure $p$, temperature field $T$ and thermodynamic pressure $p_{th}$ such that
\begin{equation*}
\begin{aligned}
    \frac{\partial \rho}{\partial t} + \nabla\cdot\left(\rho\bm{u}\right) &= 0 \\
    \rho\frac{\partial\bm{u}}{\partial t} + \rho\bm{u}\cdot\nabla\bm{u} - \nabla\cdot\boldsymbol{\tau} + \nabla p + \sigma(\bm{u} - \bm{u}_{s}) &= \rho\bm{g} \\
    \rho c_{p}\frac{\partial T}{\partial t} + \rho c_{p}\bm{u}\cdot\nabla T - \nabla\cdot\left(\kappa\nabla T\right) - \alpha T\frac{d p_{th}}{dt} &= Q
\end{aligned}
\; ,
\end{equation*}
being $c_{p}$ the specific heat coefficient at constant pressure, $\kappa$ the thermal conductivity and $\alpha$ the thermal expansion coefficient.
The system must be closed by a state equation relating $\rho$, $T$ and $p_{th}$.
Assuming an ideal gas behavior, the state equation is $\rho=p_{th}/RT$, with $R$ being the specific gas constant that relates heat capacity coefficients at constant pressure and constant volume as $R = c_{p} - c_{v}$.
Introducing the heat capacity ratio $\gamma = c_{p} / c_{v}$ allows to alternatively write the state equation for ideal gases as
\begin{equation*}
    \rho = \frac{\gamma p_{th}}{c_{p}(\gamma - 1)T}\; ,
\end{equation*}
something that at the same time makes possible to obtain the following magnitudes as
\begin{equation*}
    \frac{\partial \rho}{\partial t} = \frac{\gamma}{c_{p}(\gamma - 1)}\left(\frac{1}{T}\frac{d p_{th}}{dt}-\frac{p_{th}}{T^{2}}\frac{\partial T}{\partial t}\right)\; ,
\end{equation*}

\begin{equation*}
    \nabla \rho = -\frac{\gamma p_{th}}{c_{p}(\gamma - 1)T^{2}}\nabla T
\end{equation*}
and
\begin{equation*}
    \alpha = \left.-\frac{1}{\rho}\frac{\partial \rho}{\partial T}\right|_{p} = \frac{1}{T}\;.
\end{equation*}

The determination of the thermodynamic pressure depends on the nature of the flow, which can be roughly divided in \emph{open} and \emph{closed} flows.
For open flows (\ie flows with Neumann boundary condition) $p_{th}$ is given at each time step by the boundary conditions.
For closed flows (\ie flows without Neumann boundary condition) of ideal gases with neither inflow nor outflow conditions $p_{th}$ can be obtained from the $T$ field as
\begin{equation*}
    p_{th} = p_{{th}_{0}}\frac{\int_{\Omega}\frac{1}{T_{0}}}{\int_{\Omega}\frac{1}{T}}\; ,
\end{equation*}
where the subscript $0$ indicates the initial magnitudes at $t=0$ and $\Omega$ the problem domain.

## Variational form
The variational form (i.e. the functional to be automatically differenctiated) is obtained by multiplying previous equations by the corresponding pressure, velocity and temperature test functions $q$, $\mathbf{v}$ and $w$. After integration by parts, this yields

\begin{equation*}
\begin{aligned}
    - \left(q, \frac{\partial \rho}{\partial t}\right)_{\Omega} &- \left(q, \nabla\cdot(\rho\mathbf{u})\right)_{\Omega} - \left(\mathbf{v}, \rho\frac{\partial\mathbf{u}}{\partial t}\right)_{\Omega} - \left(\mathbf{v}, \rho\mathbf{a}\cdot\nabla\mathbf{u}\right)_{\Omega} - \left(\nabla^{s}\mathbf{v}, \boldsymbol{\tau}\right)_{\Omega} + \left(\nabla\cdot\mathbf{v}, p\right)_{\Omega} \\
    &- \left(\mathbf{v}, \sigma(\mathbf{u} - {\mathbf{u}_{s}})\right)_{\Omega} + \langle\mathbf{v}, \mathbf{t}\rangle_{\Gamma} + \left(\mathbf{v}, \rho\mathbf{g}\right)_{\Omega} + \left(w, Q\right)_{\Omega} - \left(w, \rho c_{p} \frac{\partial T}{\partial t}\right)_{\Omega} \\
    &- \left(w, \rho c_p \mathbf{a}\cdot\nabla T\right)_{\Omega} - \left(\nabla w, \kappa\nabla T\right)_{\Omega} + \langle w, \kappa\nabla T\cdot \mathbf{n}\rangle_{\Gamma} + \left(w, \alpha T \frac{d p_{th}}{dt}\right)_{\Omega} = 0\; ,
\end{aligned}
\end{equation*}
being $\mathbf{a}$ the convective velocity, which computed from the linearised previous iteration velocity $\mathbf{u}^{k-1}$ and the mesh velocity $\mathbf{u}_{m}$ as
\begin{equation*}
\mathbf{a} = \mathbf{u}^{k-1} - \mathbf{u}_{m}\; .
\end{equation*}

## Variational MultiScales stabilization
The scales separation reads

\begin{equation*}
\begin{aligned}
    p &= p_{h} + \tilde{p} \\
    \mathbf{u} &= \mathbf{u}_{h} + \tilde{\mathbf{u}} \\
    T &= T_{h} + \tilde{T}
\end{aligned}\; ,
\end{equation*}

being $p_{h}$, $\mathbf{u}_{h}$ and $T_{h}$ the finite element scales and $\tilde{p}$, $\tilde{\mathbf{u}}$ and $\tilde{T}$ the subgrid scales, which are modelled from the finite element residuals as

\begin{equation*}
\begin{aligned}
\tilde{p} & := \tau_{c} R_{c}(\mathbf{u}_{h},T_{h}) \\
\tilde{\mathbf{u}} & := \tau_{m} \mathbf{R}_{m}(p_{h}, \mathbf{u}_{h}, T_{h}) \\
\tilde{T} & := \tau_{e} R_{e}(\mathbf{u}_{h}, T_{h}) \\
\end{aligned}\; ,
\end{equation*}

being $\tau_{c}$, $\tau_{m}$ and $\tau_{e}$ the stabilization parameters which are equal to

\begin{equation*}
\begin{aligned}
    \tau_{c} &:= \frac{\mu}{\rho} + \frac{c_{2}}{c_{1}}h\lVert u\rVert +\frac{c_{3}}{c_{1}}\frac{\sigma h}{\rho} \\
    \tau_{u} &:= \left(c_{1}\frac{\mu}{h^2} + c_{2}\frac{\rho\lVert u\rVert}{h} + c_{3} \frac{\sigma}{h}\right)^{-1} \\
    \tau_{e} &:= \left(c_{1}\frac{\kappa}{h^2} + c_{2}\frac{\rho c_{p}\lVert u\rVert}{h}\right)^{-1}
\end{aligned} \; .
\end{equation*}

$c_{1}$, $c_{2}$, and $c_{3}$ are algorithmic constants defaulted in the header file to $4$, $2$, and $2$, respectively. $h$ the characteristic element size.

The finite element residuals $R^{c}(\mathbf{u}_{h}, T_{h})$, $\mathbf{R}^{m}(p_{h}, \mathbf{u}_{h}, T_{h})$ and $R^{e}(p_{h}, \mathbf{u}_{h}, T_{h})$ are defined as

\begin{equation*}
\begin{aligned}
    &R_{c}(\mathbf{u}_{h}, T_{h}) = -\frac{\partial\rho}{\partial t} + \alpha\nabla T_{h}\cdot\mathbf{u}_{h} - \rho\nabla\cdot\mathbf{u}_{h}\\
    &\mathbf{R}_{m}(p_{h}, \mathbf{u}_{h}, T_{h}) = \rho\mathbf{g} - \rho\frac{\partial\mathbf{u}_{h}}{\partial t} - \rho\mathbf{a}\cdot\nabla\mathbf{u}_{h} - \nabla p - \sigma(\mathbf{u}_{h} - \mathbf{u}_{s})\\
    &R_{e}(p_{h}, \mathbf{u}_{h}, T_{h}) = Q - \rho c_p \frac{\partial T}{\partial t} - \rho c_p \mathbf{u}_{h}\nabla T_{h} + \alpha T_{h} \frac{d p_{th}}{dt}
\end{aligned}
\end{equation*}

Once the subscales are defined, one can derive the stabilization terms to be added to the Galerkin functional. Hence, after doing the required integration by parts to remove the subscale derivatives and dropping the high order terms (the use of linear elements is assumed), one obtains the final functional to which the automatic differentiation is to be applied

\begin{equation*}
\begin{aligned}
    - \left(q, \frac{\partial \rho}{\partial t}\right)_{\Omega} &- \left(q, \nabla\rho\cdot\mathbf{u}_{h}\right)_{\Omega} - \left(q, \rho\nabla\cdot\mathbf{u}_{h}\right)_{\Omega} + \left(\nabla q, \rho\tilde{\mathbf{u}}\right)_{\Omega} - \left(\mathbf{v}, \rho\frac{\partial\mathbf{u}_{h}}{\partial t}\right)_{\Omega} - \left(\mathbf{v}, \rho\mathbf{a}\cdot\nabla\mathbf{u}_{h}\right)_{\Omega} \\
    &+ \left((\nabla\rho\cdot\mathbf{a})\mathbf{v}, \tilde{\mathbf{u}}\right)_{\Omega} + \left((\rho\nabla\cdot\mathbf{a})\mathbf{v}, \tilde{\mathbf{u}} \right)_{\Omega} + \left(\rho\mathbf{a}\cdot\nabla\mathbf{v}, \tilde{\mathbf{u}}\right)_{\Omega} - \left(\nabla^{s}\mathbf{v}, \boldsymbol{\tau}\right)_{\Omega} \\
    &+ \left(\nabla\cdot\mathbf{v}, p_{h}\right)_{\Omega} + \left(\nabla\cdot\mathbf{v}, \tilde{p}\right)_{\Omega} - \left(\mathbf{v}, \sigma(\mathbf{u}_{h} + \tilde{\mathbf{u}} - {\mathbf{u}_{s}})\right)_{\Omega} + \left(\mathbf{v}, \rho\mathbf{g}\right)_{\Omega} \\
    &+ \left(w, Q\right)_{\Omega} - \left(w, \rho c_{p} \frac{\partial T_{h}}{\partial t}\right)_{\Omega} - \left(w, \rho c_p \mathbf{a}\cdot\nabla T_{h}\right)_{\Omega} + \left((\nabla w)\rho c_p \mathbf{a}, \tilde{T}\right)_{\Omega} \\
    &+ \left(wc_p (\nabla\rho\cdot\mathbf{a}), \tilde{T} \right)_{\Omega} + \left(w\rho c_p (\nabla\cdot\mathbf{a}), \tilde{T}\right)_{\Omega} - \left(\nabla w, \kappa\nabla T_h\right)_{\Omega} + \left(w, \alpha (T_h + \tilde{T}) \frac{d p_{th}}{dt}\right)_{\Omega} = 0
\end{aligned}
\end{equation*}

Note that the quasi-static subscales assumption is taken so all the subscales time derivatives are dropped. Also note that $\rho$ (and its spatial and time derivatives) is expressed as a function of the unknown variables $T_{h}$ and $p_{th}$ by means of the equation of state above.

## IMPLEMENTATION
The remaining implementation is similar to that of the weakly compressible Navier-Stokes element.
